In [1]:
### Part of Harmonization Trial 3

import numpy as np
import pandas as pd

In [2]:
hdf = pd.HDFStore('/data/sushen/marinedebris/project/dataset_original_classes.h5', mode = 'r')
df_train = hdf.select('train')
hdf.close()

In [37]:
df_train.sample(5)

,Class,Conf,B1,B2,B3,B4,B5,B6,B7,B8,...,NDWI,NRD,NDMI,BSI,con,dis,homo,ener,cor,asm
36,7.0,1.0,1196.0,870.0,552.0,285.0,226.0,193.0,191.0,137.0,...,0.602322,-148.0,0.465241,-0.500745,1.872329,1.046474,0.558368,0.238197,0.505502,0.056738
205,7.0,1.0,1262.0,970.0,726.0,360.0,291.0,200.0,187.0,155.0,...,0.648127,-205.0,0.666667,-0.484169,2.261886,1.105636,0.559696,0.250593,0.202570,0.062797
2,7.0,1.0,1554.0,1246.0,935.0,613.0,526.0,478.0,484.0,402.0,...,0.398654,-211.0,0.294686,-0.329032,1.446848,0.897703,0.605818,0.245636,0.493143,0.060337
9,15.0,1.0,1299.0,932.0,563.0,338.0,287.0,258.0,241.0,229.0,...,0.421717,-109.0,0.817460,-0.525624,1.248531,0.821448,0.631513,0.267950,0.565594,0.071797
14,6.0,1.0,1450.0,1136.0,764.0,448.0,377.0,348.0,337.0,305.0,...,0.429373,-143.0,0.323210,-0.409291,0.288061,0.277911,0.862059,0.519251,0.639065,0.269621


In [26]:
# Old Classes
# '0': 'Unresolved error during prediction pipeline',
# '1': 'Marine Debris',
# '2': 'Dense Sargassum',
# '3': 'Sparse Sargassum',
# '4': 'Natural Organic Material',
# '5': 'Ship',
# '6': 'Clouds',
# '7': 'Marine Water',
# '8': 'Sediment-Laden Water',
# '9': 'Foam',
# '10': 'Turbid Water',
# '11': 'Shallow Water',
# '12': 'Waves',
# '13': 'Cloud Shadows',
# '14': 'Wakes',
# '15': 'Mixed Water'


# New classes 
# 1. Marine debris: Marine debris (Old 1)
# 2. Sargassum: Sargassums together (Old 2, 3)
# 3. Natural organic material: Natural organic material (Old 4)
# 4. Water: All waters together + sediment water (Old 7, 8, 10, 11, 15)
# 5. Waves: Wave, foam, wakes together (Old 9, 12, 14)
# 6. Cloud and cloud shadows (Old 6, 13)
# 7. Ship (Old 5)


In [27]:
new_cat_mapping = { '1': 'Marine Debris',
                '2': 'Sargassum',
                '3': 'Sargassum',
                '4': 'Natural Organic Material',
                '5': 'Ship',
                '6': 'Cloud & Shadows',
                '7': 'Water',
                '8': 'Water',
                '9': 'Waves',
                '10': 'Water',
                '11': 'Water',
                '12': 'Waves',
                '13': 'Cloud & Shadows',
                '14': 'Waves',
                '15': 'Water'}

def new_cat_map(x):
    return new_cat_mapping[x]

new_cat_mapping_vec = np.vectorize(new_cat_map)

In [28]:
df_train['Class'] = df_train['Class'].apply(lambda x: new_cat_mapping_vec(str(int(x))))

In [29]:
df_train.sample(5)

,Class,Conf,B1,B2,B3,B4,B5,B6,B7,B8,...,NDWI,NRD,NDMI,BSI,con,dis,homo,ener,cor,asm
15,Cloud & Shadows,1.0,1291.0,808.0,507.0,309.0,258.0,237.0,226.0,188.0,...,0.458993,-121.0,0.314685,-0.419815,1.374733,0.666667,0.723203,0.282856,0.849510,0.080008
39,Water,1.0,1178.0,889.0,593.0,306.0,263.0,230.0,217.0,179.0,...,0.536269,-127.0,0.415020,-0.475138,6.323184,1.475962,0.613544,0.322767,0.679554,0.104179
23,Ship,1.0,1723.0,1769.0,1542.0,1436.0,1478.0,1294.0,1458.0,1271.0,...,0.096338,-165.0,0.262792,-0.165197,4.447516,1.156384,0.651706,0.299017,0.792053,0.089411
24,Water,1.0,1246.0,917.0,611.0,330.0,272.0,227.0,201.0,165.0,...,0.574742,-165.0,0.520737,-0.478142,1.881143,1.027244,0.571316,0.275783,0.098527,0.076056
136,Water,1.0,1263.0,961.0,712.0,360.0,283.0,211.0,185.0,153.0,...,0.646243,-207.0,0.690608,-0.483356,2.268830,1.138755,0.540764,0.240868,0.145857,0.058017


In [30]:
df_train['Class'].value_counts()

Water                       91307
Cloud & Shadows             42950
Waves                       27574
Ship                        23913
Marine Debris               23456
Sargassum                   11604
Natural Organic Material     2790
Name: Class, dtype: int64

In [31]:
new_dataset_path = '/data/sushen/marinedebris/project/dataset_trial_3_seven_classes.h5'
hdf_new = pd.HDFStore(new_dataset_path, mode = 'w')
hdf_new.append('train', df_train, format='table', data_columns=True)
hdf_new.close()

In [32]:
hdf_test = pd.HDFStore(new_dataset_path, mode = 'r')
df_trial = hdf_test.select('train')
hdf_test.close()
df_trial.sample(5)

,Class,Conf,B1,B2,B3,B4,B5,B6,B7,B8,...,NDWI,NRD,NDMI,BSI,con,dis,homo,ener,cor,asm
55,Water,1.0,1402.0,1064.0,769.0,477.0,410.0,357.0,352.0,315.0,...,0.418819,-162.0,0.285714,-0.357952,3.895032,1.201389,0.604392,0.264443,0.734433,0.069930
13,Waves,1.0,1302.0,1078.0,771.0,500.0,391.0,363.0,344.0,362.0,...,0.360989,-138.0,0.348231,-0.361702,1.869792,1.015358,0.576068,0.212678,0.570871,0.045232
15,Water,1.0,1322.0,927.0,556.0,363.0,315.0,287.0,280.0,232.0,...,0.411168,-131.0,0.189744,-0.379762,1.702991,1.007479,0.565359,0.266155,0.353944,0.070838
23,Ship,1.0,1256.0,970.0,783.0,1234.0,1067.0,1237.0,1279.0,1246.0,...,-0.228191,12.0,0.006869,0.052789,5.327858,1.739450,0.436431,0.142313,0.729402,0.020253
87,Waves,1.0,1504.0,1499.0,986.0,754.0,735.0,669.0,565.0,775.0,...,0.119818,21.0,0.211884,-0.287656,3.387553,1.366720,0.497278,0.165331,0.729005,0.027334


In [33]:
df_trial['Class'].value_counts()

Water                       91307
Cloud & Shadows             42950
Waves                       27574
Ship                        23913
Marine Debris               23456
Sargassum                   11604
Natural Organic Material     2790
Name: Class, dtype: int64